In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from model_utils import *
from stats_tests import *

In [ ]:
data_path = "/home/angps/Documents/Thesis/Data/"

In [ ]:
df = pd.read_csv(data_path + 'location_map.csv')
df.head()

In [2]:
from collections import defaultdict
l = df['Period'].min()
h = df['Period'].max()
dic = dict()
ct = 0
for g in df['geohash6'].unique():
    temp = df.groupby(["geohash6","Period"])['demand'].mean()[g].to_dict()
    temp = defaultdict(int, temp)
    dic[g] = np.array([temp[i] for i in range(h+1)])
    ct += 1
    if ct % 100 == 0:
        print(ct)

NameError: name 'df' is not defined

In [ ]:
final_df = pd.DataFrame.from_dict(dic, orient='index')
final_df = final_df.reset_index()
final_df = final_df.rename(columns={"index":"loc"})
final_df.head()

In [ ]:
final_df.shape

In [ ]:
# final_df.to_csv(data_path + "Sample/final_df.csv", index=False)

**Visualisation**

In [ ]:
# import geohash2
# final_df['latitude'] = final_df['loc'].apply(lambda x: float(geohash2.decode_exactly(x)[0]))
# final_df['longitude'] = final_df['loc'].apply(lambda x: float(geohash2.decode_exactly(x)[1]))
# final_df.to_csv(data_path + "Sample/final_df.csv", index=False)

In [ ]:
retail_data = pd.read_csv(data_path + 'retail_data.csv')
retail_data.head()

In [ ]:
df = pd.read_csv(data_path + '/Sample/final_df.csv')
df.head()

# **Clustering**

**Cluster by location**

In [ ]:
plt_elbow_for_clustering(df, col_names=['latitude','longitude'])

In [ ]:
labels = cluster_geog(df, col_names=['latitude','longitude'], n_clusters = 5)

In [ ]:
plt_geog_cluster(df, col_names=['latitude','longitude'], labels=labels, map_img=plt.imread(data_path + 'images/sg_map.png'))

In [ ]:
plt_elbow_for_clustering(retail_data, col_names=['latitude','longitude'])

In [ ]:
labels = cluster_geog(retail_data, col_names=['latitude','longitude'], n_clusters = 5)

In [ ]:
plt_geog_cluster(retail_data, col_names=['latitude','longitude'], labels=labels, map_img=plt.imread(data_path + 'images/sg_map.png'))

**Cluster by time series**

In [ ]:
#Cluster by correlation
ts_df = df.iloc[:,1:-2]
grps = cluster_ts(ts_df, n_clusters=6, dist_metric='correlation')

#Cluster by DTW



In [ ]:
cluster_loc = get_cluster_loc(grps)

**Combine cluster groups**

In [ ]:
#Create testing df for easy testing
sample_df = pd.read_csv(data_path + '/Sample/sample.csv')
sample_df.head()

In [ ]:
sample_df['cluster'] = ['Cluster_0', 'Cluster_1', 'Cluster_2','Cluster_1','Cluster_0']
sample_df

In [ ]:
sample_cluster_loc = {0:[0,4], 1:[1,3], 2:[2]}
sample_pred = pd.DataFrame.from_dict({"Cluster_0":[10,20], "Cluster_1":[100, 50], "Cluster_2":[30,10]}, orient="index")
sample_pred

In [ ]:
t = sample_df.groupby("cluster").sum()
#t.loc['Cluster_1','1']
t

In [ ]:
form_cluster_grp(sample_df, [0,1,2,1,0])

In [3]:
ts_cluster_df = form_cluster_grp(ts_df, grps)
ts_cluster_df

NameError: name 'form_cluster_grp' is not defined

**Prediction Model**

**Split training-test**

In [ ]:
from statsmodels.tsa.api import VAR

In [ ]:
# requires dataframe to have time as rows and locs/vars in columns
model = VAR(ts_cluster_df.T)

In [ ]:
model.select_order().selected_orders


In [ ]:
results = model.fit(14)

In [ ]:
results.summary()

In [ ]:
results.plot()

In [ ]:
results.plot_acorr()

In [ ]:
lag_order = results.k_ar

In [ ]:
results.forecast(ts_cluster_df.T.values[-lag_order:], 5)

In [ ]:
results.plot_forecast(10)

In [ ]:
irf = results.irf(14)
plt.figure(figsize = (24,24))
irf.plot(orth=False)

In [ ]:
irf.plot(impulse='Cluster_0')

In [ ]:
irf.plot_cum_effects(orth=False)

In [ ]:
fevd = results.fevd(5)
fevd.summary()

In [ ]:
results.fevd(20).plot()

In [ ]:
results.test_normality().summary()

**Assign total demand of each cluster to individual locations**

In [26]:
ts_cluster_df

,0,1,2,3,4,5,6,7,8,9,...,5846,5847,5848,5849,5850,5851,5852,5853,5854,5855
cluster,,,,,,,,,,,,,,,,,,,,,
Cluster_0,30.018154,33.148272,35.997936,38.671355,43.986918,49.033977,55.672367,58.417889,61.826454,66.41204,...,15.182077,14.876712,16.053734,17.194969,17.272335,17.687096,20.729729,23.798470,28.051767,30.447158
Cluster_1,0.208278,0.235666,0.193818,0.212962,0.272393,0.244445,0.257339,0.183418,0.197991,0.30977,...,1.122574,0.648463,0.616607,0.462037,0.310822,0.256371,0.258999,0.271552,0.242219,0.302805
Cluster_2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Cluster_3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Cluster_4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Cluster_5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [27]:
ts_df.head()

,0,1,2,3,4,5,6,7,8,9,...,5847,5848,5849,5850,5851,5852,5853,5854,5855,cluster
0,0.054858,0.086209,0.050739,0.075174,0.062867,0.056765,0.069417,0.079853,0.119192,0.115955,...,0.027943,0.029310,0.033446,0.018348,0.045894,0.068427,0.034809,0.059814,0.045901,Cluster_0
1,0.000000,0.005546,0.013577,0.004720,0.004425,0.000303,0.013996,0.004471,0.014096,0.005484,...,0.004884,0.000000,0.000000,0.001958,0.000778,0.000000,0.000000,0.000000,0.000000,Cluster_0
2,0.022881,0.019733,0.023053,0.029018,0.073814,0.066341,0.061355,0.094834,0.086832,0.071593,...,0.011049,0.015564,0.000000,0.013121,0.000000,0.005243,0.014848,0.007184,0.035813,Cluster_0
3,0.012395,0.048042,0.078544,0.074393,0.117350,0.092151,0.061261,0.083871,0.083997,0.074020,...,0.000000,0.000000,0.000000,0.000000,0.002199,0.030536,0.038095,0.058625,0.021198,Cluster_0
4,0.000000,0.000000,0.000000,0.011483,0.003363,0.015389,0.016686,0.017399,0.004562,0.025001,...,0.015546,0.000000,0.002288,0.011023,0.008362,0.000000,0.001531,0.000000,0.000000,Cluster_0


In [28]:
sample_df

,1,2,3,4,5,6,cluster
0,2,4,5,1,2,3,Cluster_0
1,4,8,9,1,1,6,Cluster_1
2,2,2,9,3,5,4,Cluster_2
3,3,5,4,2,6,1,Cluster_1
4,1,2,4,7,5,1,Cluster_0


In [ ]:
def get_past_distr(train_df, cluster_loc):
    train_df_sum = train_df.sum(axis=1)
    distr_dict = dict()
    num_clusters = len(cluster_loc.keys())
    for clust in range(num_clusters):
        distr_dict[clust] = list()
        for locs in cluster_loc[clust]:
            distr_dict[clust].append(train_df_sum[locs])
        if sum(distr_dict[clust]) > 0:
            distr_dict[clust] = np.divide(distr_dict[clust], sum(distr_dict[clust]))
    return distr_dict

In [29]:
d = get_past_distr(sample_df, sample_cluster_loc)
d

{0: array([0.45945946, 0.54054054]), 1: array([0.58, 0.42]), 2: array([1.])}

In [30]:
assign_demand_to_loc(sample_pred, d, sample_cluster_loc)

,0,1
0,4.594595,9.189189
1,58.000000,29.000000
2,30.000000,10.000000
3,42.000000,21.000000
4,5.405405,10.810811


**Get results**